In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import eigh

In [4]:
ksi_1 = np.arange(-1,1.1,0.1)
ksi_2 = np.arange(-1,1.1,0.1)
m = 0.067
h = 1
w = 10/27211.6
hw = h*w

def initializing(NN):
    nlg=np.zeros((NN**2, 4))
    for j in  range(0, NN):
        for i in range(0, NN):
            nlg[i+(j*NN)][0]=i+(j*NN)+j        + 1   #lewy  dolny
            nlg[i+(j*NN)][1]=i+(j*NN)+j+1      + 1   #prawy dolny
            nlg[i+(j*NN)][2]=i+(j*NN)+j+1+NN   + 1   #lewy  gorny
            nlg[i+(j*NN)][3]=i+(j*NN)+j+1+NN+1 + 1   #prawy gorny
    nlg = nlg.astype(int)
    return nlg
    
def g(i,ksi1,ksi2):
    if i == 0:
        return (1 - ksi1)/2 * (1 - ksi2)/2
    if i == 1:
        return (1 + ksi1)/2 * (1 - ksi2)/2
    if i == 2:
        return (1 - ksi1)/2 * (1 + ksi2)/2
    if i == 3:
        return (1 + ksi1)/2 * (1 + ksi2)/2
        
def dg(i,f, ksi1, ksi2):
    d = 0.001
    if f == 1:
        return (g(i,ksi1,ksi2 + d) - g(i,ksi1,ksi2 - d))/(2*d)
    if f == 2:
        return (g(i,ksi1 + d,ksi2) - g(i,ksi1 - d,ksi2))/(2*d)
        
def brzeg(a):
    if (a<NN+1):
        return 0
    elif (a>((NN+1)**2)-(NN+2)):
        return 0
    elif (((a+1)%(NN+1))==1):
        return 0
    elif (((a+1)%(NN+1))==0):
        return 0
    else:
        return 1
        
def FUN(L, N):
    a = L/(2*N)
    NN = 2*N
    nlg = initializing(NN)
    wspx = np.zeros((NN+1, NN+1))
    for i in range(NN+1):
        for j in range(NN+1):
            wspx[i, j] = -(L/2) + j * a
            
    wspy = np.zeros((NN+1, NN+1))
    for i in range(NN+1):
        for j in range(NN+1):
            wspy[i, j] = -(L/2) + i * a

    psi_nlg=np.zeros((4*N*N, 4))
    for i in range(0, 4*N*N):
        for j in range(0, 4):
            psi_nlg[i][j]=np.exp(-m*w/(2*h)*(wspx[int((nlg[i][j]-1)/(NN+1))][int((nlg[i][j]-1)%(NN+1))]**2 + wspy[int((nlg[i][j]-1)/(NN+1))][int((nlg[i][j]-1)%(NN+1))]**2))
    coordinates = np.zeros(((NN+1)**2,2))
    for i in range(NN+1):
        for j in range(NN+1):
            coordinates[(NN+1)*i+j][0] = wspx[i][j]
            coordinates[(NN+1)*i+j][1] = wspy[i][j]

    wx = [5/9, 8/9, 5/9]
    px = [-np.sqrt(3/5), 0, np.sqrt(3/5)]
    S_k = np.zeros((4,4))
    
    
    for i in range(4):
        for j in range(4):
            G = 0
            for k in range(3):
                for l in range(3):
                    G += wx[k] * wx[l] * g(i,px[l],px[k]) * g(j,px[l],px[k])
            S_k[i,j] = G * a**2/4
            
    T_k = np.zeros((4,4))
    
    for i in range(4):
        for j in range(4):
            DG = 0
            for k in range(3):
                for l in range(3):
                    DG += wx[k] * wx[l] * (dg(i,1,px[l],px[k]) * dg(j,1,px[l],px[k]) + dg(i,2,px[l],px[k]) * dg(j,2,px[l],px[k]))
            T_k[i,j] = DG * h**2 / (2*m)
    V_k = np.zeros((NN**2,4,4))
    V_nlg = np.zeros((4,4))
    for n in range(NN**2):
        for i in range(4):
            for j in range(4):
                V = 0
                for k in range(3):
                    for l in range(3):
                        x = 0.5 * (coordinates[(nlg[n][0] - 1)][0] * (1 - px[k]) + coordinates[(nlg[n][1] - 1)][0] * (1 + px[k]))
                        y = 0.5 * (coordinates[(nlg[n][1] - 1)][1] * (1 - px[l]) + coordinates[(nlg[n][2] - 1)][1] * (1 + px[l]))
    
                        V += wx[k] * wx[l] * (x**2 + y**2) * g(i,px[k],px[l]) * g(j,px[k],px[l]) 
                V_nlg[i,j] = V * a**2 / 4 * hw**2 *m / 2
        V_k[n] = V_nlg
    S = np.zeros(((NN+1)**2,(NN+1)**2))
    H = np.zeros(((NN+1)**2,(NN+1)**2))
    
    for k in range(NN**2):
        for i1 in range(4):
            for i2 in range(4):
                S[nlg[k][i1]-1,nlg[k][i2]-1] += S_k[i1,i2]
                H[nlg[k][i1]-1,nlg[k][i2]-1] += T_k[i1,i2] + V_k[k,i1,i2]
    for i in range(0, (NN+1)**2):
        if brzeg(i)==0: 
            for j in range(0, (NN+1)**2):
                S[i][j]=0
                S[j][i]=0
                H[i][j]=0
                H[j][i]=0
                S[i][i]=1
                H[i][i]=-1410

    return eigh(H, S)

In [5]:
N = 10
NN = 2*N
L = 100/0.05292
# energies, eigenvectors = FUN(L, N)
# wektor=np.zeros((NN+1)**2)
# for i in range(0, (NN+1)**2):
#     wektor[i]=eigenvectors[i][((NN+1)**2-1)-(NN-1)**2+4]

In [6]:
# num = np.size(energies) - ((NN+1)**2)-(NN-1)**2
# Energy = np.zeros(num)
# for i in range(num):
#     Energy = energies.append([((NN+1)**2) - (NN-1)**2 + i])
# Energy
N_loop = np.linspace(1,15,5)
l1 = 50
l2 = 250
L_loop = np.linspace(l1,l2,10)/0.05292

In [7]:
pp = 2137
L_ = 0
N_ = 0
for i in range(N_loop.shape[0]):
    NN = N_loop[i] * 2
    for j in range(L_loop.shape[0]):
        energies, eigenvectors = FUN(L_loop[j], int(N_loop[i]))
        if energies[int(((NN+1)**2-1)-(NN-1)**2+1)] < pp:
            pp = energies[int(((NN+1)**2-1)-(NN-1)**2+1)]
            L_ = L_loop[j]
            N_ = N_loop[i]
            

In [9]:
print(L_)
print(N_)
print(L_*0.05292)

1364.7434282354918
15.0
72.22222222222223
